# skills walkthrough（细讲版）

## 目标与先修
- 目标：理解 skill 路由如何从 catalog 中选择候选技能并生成执行计划。
- 先修：掌握字典字段读取、列表过滤、基础路由评分概念。


## 流程总览
1. 加载 skill catalog。
2. 请求模型路由结果并观察原始 JSON。
3. 提取命中技能和执行计划。
4. 用 `run_demo` 验证 trace。


### 环境检查（在线模块）
- 本步做什么：检查 `DEEPSEEK_API_KEY` 是否存在。
- 为什么这样做：`skills` 依赖在线 LLM API；缺 key 时必须显式失败，避免误以为流程成功。
- 输入：环境变量 `DEEPSEEK_API_KEY`。
- 输出：通过则继续执行；失败则抛出 `RuntimeError`。
- 观察点：失败信息要明确提示“这是配置问题，不是业务逻辑问题”。


In [ ]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY", "")
if not api_key:
    raise RuntimeError("缺少 DEEPSEEK_API_KEY：在线 walkthrough 不能继续执行。")

print("环境检查通过：已检测到 DEEPSEEK_API_KEY")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 导入函数并准备问题
- 本步做什么：导入 catalog、路由与 demo 函数，并定义用户请求。
- 为什么这样做：统一输入，方便比较分步与端到端结果。
- 输入：`user_text`。
- 输出：待路由的查询文本。
- 观察点：输入应包含明确意图，便于触发某个技能。


In [ ]:
from component import load_skill_catalog, request_skill_routing, run_demo

user_text = "请帮我创建一个新技能，用于规范化代码评审流程。"
print(user_text)


### Step 2: 加载 skill catalog
- 本步做什么：调用 `load_skill_catalog()`。
- 为什么这样做：路由前必须先知道可选技能集合。
- 输入：无。
- 输出：技能定义列表。
- 观察点：关注技能名、触发条件、能力描述。


In [ ]:
catalog = load_skill_catalog()
print(f"catalog size: {len(catalog)}")
show_json(catalog)


### Step 3: 请求路由结果（原始）
- 本步做什么：调用 `request_skill_routing(user_text, catalog)`。
- 为什么这样做：先看模型给出的原始路由 JSON，再做字段提取。
- 输入：`user_text` + `catalog`。
- 输出：`routing` 字典。
- 观察点：重点看 `matched`、`selected_name`、`score`、`execution_plan`。


In [ ]:
routing = request_skill_routing(user_text, catalog)
show_json(routing)


### Step 4: 提取命中技能与执行计划
- 本步做什么：从原始路由结果中抽取教学关注字段。
- 为什么这样做：把“模型原始输出”变成“可解释的执行视图”。
- 输入：`routing`。
- 输出：`selection` 与 `plan`。
- 观察点：`matched=False` 时应正确处理空选择，避免直接崩溃。


In [ ]:
selection = {
    "matched": bool(routing.get("matched", False)),
    "selected_name": routing.get("selected_name"),
    "score": routing.get("score"),
    "trigger_hits": routing.get("trigger_hits", []),
}
plan = routing.get("execution_plan", [])

show_json(selection)
print("execution_plan:")
show_json(plan)


## 端到端结果

### Step 5: 运行 run_demo 并核对 trace
- 本步做什么：调用 `run_demo(user_text)`。
- 为什么这样做：验证分步观察与端到端 trace 一致。
- 输入：`user_text`。
- 输出：`demo_result`。
- 观察点：trace 应按 catalog -> routing -> selection -> final answer 的节奏展开。


In [ ]:
demo_result = run_demo(user_text)
show_json(demo_result)

print()
print("TRACE EVENTS:")
print([event["event"] for event in demo_result["trace"]])


## 常见错误
- 缺少 `DEEPSEEK_API_KEY`：在线请求无法执行。
- `matched=False` 但代码假设一定有 `selected_name`：会导致空值处理问题。
- execution plan 非列表：需要先做类型判断再遍历。

## 总结
- skills 模块核心不是“回答内容”，而是“选择哪个能力并给出可执行计划”。
- 排错优先级：先看 `routing` 原始结果，再看 `selection/plan` 解释层。
